In [1]:
#import statements, weight definitions, data classes, rounding function
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore') 

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

weight1=[[ 0.1642,  0.2455, -0.3093, -0.4260, -0.1927],
        [ 0.1405,  0.2877,  0.1791,  0.2886,  0.0281],
        [-0.4129,  0.2434, -0.3342,  0.0638,  0.0898],
        [ 0.3933,  0.3019,  0.0625,  0.2342, -0.4222],
        [-0.3019,  0.1200, -0.1640, -0.1984, -0.1032]]

weight2=[[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0]]

weight4=[[-0.0068,  0.1320, -0.4003,  0.0965, -0.4061],
        [ 0.2241,  0.1185,  0.0251, -0.0104, -0.0224],
        [ 0.4361, -0.0875,  0.0647,  0.3484,  0.4034],
        [ 0.1191, -0.3272,  0.2531,  0.2740, -0.3526],
        [ 0.3558, -0.0614, -0.2608, -0.1395, -0.0635]]


weight5=[[-0.0115,  0.1046,  0.2259,  0.1117, -0.0206],
        [-0.4093, -0.2933, -0.3829, -0.2851,  0.1360],
        [ 0.1195,  0.3427,  0.4183,  0.1673,  0.1913],
        [-0.2937, -0.2824,  0.1436, -0.2029,  0.1594],
        [ 0.0140,  0.1943, -0.2809,  0.2599,  0.0647]]

weightout=[[.50,.50,.50,.50,.50]]
bias1=[ 0.1889, -0.4299,  0.0239,  0.3833, -0.2821]
bias2=[0.0,.0,0.0,0.0,0.0]
#weight2
#bias3=
#bias4=
#bias5=


#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

    
    
    
#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
    
    

def binary_acc(y_pred, y_test): 
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc




In [2]:
#model class 5 layers, then an output layer
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(5, 5) 
        self.layer_2 = nn.Linear(5, 5) 
        self.layer_3 = nn.Linear(5, 5)
        self.layer_4 = nn.Linear(5, 5)
        self.layer_5 = nn.Linear(5, 5) 
        self.layer_out = nn.Linear(5, 1) #output layer
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        
        self.batchnorm1 = nn.BatchNorm1d(5)
        self.batchnorm2 = nn.BatchNorm1d(5)
        self.batchnorm3 = nn.BatchNorm1d(5)
        self.batchnorm4 = nn.BatchNorm1d(5)
        self.batchnorm5 = nn.BatchNorm1d(5)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.batchnorm5(x)        
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [3]:
#6 layers then an output layer. first 5 and output are copied
class extra_inner_layer(nn.Module):
    def __init__(self, originalmodel):
        super(extra_inner_layer, self).__init__()
        self.layer_1 = copy.deepcopy(originalmodel.layer_1)
        self.layer_2 = copy.deepcopy(originalmodel.layer_2)
        self.layer_3 = copy.deepcopy(originalmodel.layer_3)
        self.layer_4 = copy.deepcopy(originalmodel.layer_4)
        self.layer_5 = copy.deepcopy(originalmodel.layer_5)
        self.layer_6 = nn.Linear(5, 5)
        self.layer_out = copy.deepcopy(originalmodel.layer_out)
        
        self.relu = copy.deepcopy(originalmodel.relu)        
        self.dropout = copy.deepcopy(originalmodel.dropout)
        
        self.batchnorm1 = copy.deepcopy(originalmodel.batchnorm1)
        self.batchnorm2 = copy.deepcopy(originalmodel.batchnorm2)
        self.batchnorm3 = copy.deepcopy(originalmodel.batchnorm3)
        self.batchnorm4 = copy.deepcopy(originalmodel.batchnorm4)
        self.batchnorm5 = copy.deepcopy(originalmodel.batchnorm5)
        self.batchnorm6 = nn.BatchNorm1d(5)
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.batchnorm5(x) 
        x = self.relu(self.layer_6(x))
        x = self.batchnorm6(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x








        
            

In [4]:
#append layer at end
class append_layer(nn.Module):
    def __init__(self, originalmodel):
        super(append_layer, self).__init__()
        self.layer_1 = copy.deepcopy(originalmodel.layer_1)
        self.layer_2 = copy.deepcopy(originalmodel.layer_2)
        self.layer_3 = copy.deepcopy(originalmodel.layer_3)
        self.layer_4 = copy.deepcopy(originalmodel.layer_4)
        self.layer_5 = copy.deepcopy(originalmodel.layer_5)
        self.layer_6 = copy.deepcopy(originalmodel.layer_out)
        self.layer_out = nn.Linear(1,1)
        
        self.relu = copy.deepcopy(originalmodel.relu)
        self.dropout = copy.deepcopy(originalmodel.dropout)
        
        self.batchnorm1 = copy.deepcopy(originalmodel.batchnorm1)
        self.batchnorm2 = copy.deepcopy(originalmodel.batchnorm2)
        self.batchnorm3 = copy.deepcopy(originalmodel.batchnorm3)
        self.batchnorm4 = copy.deepcopy(originalmodel.batchnorm4)
        self.batchnorm5 = copy.deepcopy(originalmodel.batchnorm5)
        self.batchnorm6 = nn.BatchNorm1d(1)
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.batchnorm5(x) 
        x = self.relu(self.layer_6(x))
        x = self.batchnorm6(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x


        





In [5]:
#append layer and output
class append_layer_and_ouput(nn.Module):
    def __init__(self, originalmodel):
        super(append_layer_and_ouput, self).__init__()
        self.layer_1 = copy.deepcopy(originalmodel.layer_1)
        self.layer_2 = copy.deepcopy(originalmodel.layer_2)
        self.layer_3 = copy.deepcopy(originalmodel.layer_3)
        self.layer_4 = copy.deepcopy(originalmodel.layer_4)
        self.layer_5 = copy.deepcopy(originalmodel.layer_5)
        self.layer_6 = copy.deepcopy(originalmodel.layer_out)
        self.layer_7 = nn.Linear(1,5)
        self.layer_out = nn.Linear(5,1)
        
        self.relu = copy.deepcopy(originalmodel.relu)
        self.dropout = copy.deepcopy(originalmodel.dropout)
        
        
        
        self.batchnorm1 = copy.deepcopy(originalmodel.batchnorm1)
        self.batchnorm2 = copy.deepcopy(originalmodel.batchnorm2)
        self.batchnorm3 = copy.deepcopy(originalmodel.batchnorm3)
        self.batchnorm4 = copy.deepcopy(originalmodel.batchnorm4)
        self.batchnorm5 = copy.deepcopy(originalmodel.batchnorm5)
        self.batchnorm6 = nn.BatchNorm1d(1)
        self.batchnorm7 = nn.BatchNorm1d(5)
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.batchnorm5(x) 
        x = self.relu(self.layer_6(x))
        x = self.batchnorm6(x)
        x = self.relu(self.layer_7(x))
        x = self.batchnorm7(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x
        
        
        




In [6]:
#groundtruth model
#df = pd.read_csv("Dataset_spine.csv")
df= pd.read_csv("Breast_cancer_data.csv")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




X = df.iloc[:, 0:-1] #input columns
scaler = StandardScaler()
X_allscaled = scaler.fit_transform(X) #scales data
X_formatted = testData(torch.FloatTensor(X_allscaled)) #format data for input
Truth_loader = DataLoader(dataset=X_formatted, batch_size=1) #format data for input



GroundTruth = binaryClassification()
#set weights to "evenly distributed" 0, 1
GroundTruth.layer_1.weight=torch.nn.Parameter(data=torch.tensor(weight1), requires_grad=True)
GroundTruth.layer_2.weight=torch.nn.Parameter(data=torch.tensor(weight2), requires_grad=True)
GroundTruth.layer_3.weight=torch.nn.Parameter(data=torch.tensor(weight2), requires_grad=True)
GroundTruth.layer_4.weight=torch.nn.Parameter(data=torch.tensor(weight4), requires_grad=True)
GroundTruth.layer_5.weight=torch.nn.Parameter(data=torch.tensor(weight5), requires_grad=True)
GroundTruth.layer_out.weight=torch.nn.Parameter(data=torch.tensor(weightout), requires_grad=True)

GroundTruth.layer_1.bias=torch.nn.Parameter(data=torch.tensor(bias1), requires_grad=True)
GroundTruth.layer_2.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_3.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_4.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_5.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_out.bias=torch.nn.Parameter(data=torch.tensor([-3.0]),requires_grad=True)

#print(GroundTruth.layer_1.weight)

truth_list = []
GroundTruth.eval()
with torch.no_grad():
    for X_batch in Truth_loader:
        X_batch = X_batch.to(device)
        y_truth = GroundTruth(X_batch)
        y_truth = torch.sigmoid(y_truth)
        y_truthtag = torch.round(y_truth)
        truth_list.append(y_truthtag.cpu().numpy())

y = [a.squeeze().tolist() for a in truth_list] #new truth values

#split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)
#rescale data 
scaler = StandardScaler()



#ssplit data
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=.5, random_state=69)
X_train=scaler.fit_transform(X_train)                                                        
X_train1 = scaler.fit_transform(X_train1)
X_train2=scaler.fit_transform(X_train2)
X_test = scaler.transform(X_test)


                                                          
                                                      

EPOCHS = 50 #number of passes of whole data
BATCH_SIZE = 64 #size of data going through at once
LEARNING_RATE = 0.001



train_data= trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))

train_data1= trainData(torch.FloatTensor(X_train1), 
                       torch.FloatTensor(y_train1))
                                         
train_data2= trainData(torch.FloatTensor(X_train2), 
                       torch.FloatTensor(y_train2))

test_data = testData(torch.FloatTensor(X_test))

#data loader initiation
train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
train_loader1= DataLoader(dataset=train_data1, batch_size=BATCH_SIZE, shuffle=True)
train_loader2= DataLoader(dataset=train_data2, batch_size=BATCH_SIZE, shuffle=True)                         
test_loader = DataLoader(dataset=test_data, batch_size=1)




device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [7]:
#train first model
trained = binaryClassification()
trained.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(trained.parameters(), lr=LEARNING_RATE)

#print(list(model.parameters())[0])

#train trained model
trained.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = trained(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

    
    
y_fullmodelpred_list =[]
trained.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]



    
fullmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       

print("# correct: ",fullmodelcounter, " out of ", len(y_test))
        
print("Full model statistics")
print(classification_report(y_test, y_fullmodelpred_list))    
    
    
    
    


# correct:  178  out of  188
Full model statistics
              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94        88
         1.0       0.95      0.95      0.95       100

    accuracy                           0.95       188
   macro avg       0.95      0.95      0.95       188
weighted avg       0.95      0.95      0.95       188



In [8]:
#define models with extra layers
trained_append_layer=append_layer(trained)
trained_append_layeroutput=append_layer_and_ouput(trained)
trained_extra_inner_layer=extra_inner_layer(trained)

optimizerappend = optim.Adam(trained_append_layer.parameters(), lr=LEARNING_RATE)
optimizermiddle = optim.Adam(trained_append_layeroutput.parameters(), lr=LEARNING_RATE)
optimizerappendoutput = optim.Adam(trained_extra_inner_layer.parameters(), lr=LEARNING_RATE)

        

      
firstlayers_model1=binaryClassification()

In [9]:
#append layer train with first half then second

#train using first half
trained_append_layer.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader1:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizerappend.zero_grad()
        
        y_pred = trained_append_layer(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizerappend.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader1):.5f} | Acc: {epoch_acc/len(train_loader1):.3f}')


print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")

print("Results using half training data:")

#copy over first part of model
firstlayers_model1.layer_1 = copy.deepcopy(trained_append_layer.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_append_layer.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_append_layer.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_append_layer.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_append_layer.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_append_layer.layer_6)

#results full model, half data
y_fullmodelpred_list =[]
trained_append_layer.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained_append_layer(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]




#results partial model
y_partialmodelpred_list =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_partialmodelpred_list.append(y_pred_tag.cpu().numpy())


y_partialmodelpred_list = [a.squeeze().tolist() for a in y_partialmodelpred_list]




#counts correct values
fullmodelcounter=0
partialmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       
    if y_partialmodelpred_list[i]==y_test[i]:
        partialmodelcounter=partialmodelcounter+1

print("half data full model correct: ",fullmodelcounter, " out of ", len(y_test))
print("half data partial model correct: ",partialmodelcounter, " out of ", len(y_test))
        
print("Full model statistics, half data")
print(classification_report(y_test, y_fullmodelpred_list))


print("Partial model statistics, half data")
print(classification_report(y_test, y_partialmodelpred_list))



#train with second half of data set
trained_append_layer.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader2:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizerappend.zero_grad()
        
        y_pred = trained_append_layer(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizerappend.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()


firstlayers_model1.layer_1 = copy.deepcopy(trained_append_layer.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_append_layer.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_append_layer.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_append_layer.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_append_layer.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_append_layer.layer_6)



        
print("Results using full training data:")



#full model predictions
y_fullmodelpred_list =[]
trained_append_layer.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained_append_layer(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]


#partial model predictions
y_partialmodelpred_list =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_partialmodelpred_list.append(y_pred_tag.cpu().numpy())


y_partialmodelpred_list = [a.squeeze().tolist() for a in y_partialmodelpred_list]





#prints correct results
fullmodelcounter=0
partialmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       
    if y_partialmodelpred_list[i]==y_test[i]:
        partialmodelcounter=partialmodelcounter+1
        
print("full data full model correct: ",fullmodelcounter, " out of ", len(y_test))
print("full data partial model correct: ",partialmodelcounter, " out of ", len(y_test))


print("Full model statistics")
print(classification_report(y_test, y_fullmodelpred_list))


print("Partial model statistics")
print(classification_report(y_test, y_partialmodelpred_list))
        



        
    #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')
    
    

precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
Results using half training data:
half data full model correct:  100  out of  188
half data partial model correct:  89  out of  188
Full model statistics, half data
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        88
         1.0       0.53      1.00      0.69       100

    accuracy                           0.53       188
   macro avg       0.27      0.50      0.35       188
weighted avg       0.28      0.53      0.37       188

Partial model statistics, half data
              precision    recall  f1-score   support

         0.0       0.47      1.00      0.64        88
         1.0       1.00      0.01      0.02       100

    accuracy                           0.47       188
   macro avg       0.74      0.51      0.33       188
weighted avg       0.75      0.47      0.31       188

Results using full training data:
full data full model correct:  172  out o

In [10]:
#append layer and output train with first half then second

#train using first half
trained_append_layeroutput.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader1:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizerappendoutput.zero_grad()
        
        y_pred = trained_append_layeroutput(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizerappendoutput.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")

print("Results using half training data:")

#copy over first part of model
firstlayers_model1.layer_1 = copy.deepcopy(trained_append_layeroutput.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_append_layeroutput.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_append_layeroutput.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_append_layeroutput.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_append_layeroutput.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_append_layeroutput.layer_6)

#results full model, half data
y_fullmodelpred_list =[]
trained_append_layeroutput.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained_append_layeroutput(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]




#results partial model
y_partialmodelpred_list =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_partialmodelpred_list.append(y_pred_tag.cpu().numpy())


y_partialmodelpred_list = [a.squeeze().tolist() for a in y_partialmodelpred_list]




#counts correct values
fullmodelcounter=0
partialmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       
    if y_partialmodelpred_list[i]==y_test[i]:
        partialmodelcounter=partialmodelcounter+1

print("half data full model correct: ",fullmodelcounter, " out of ", len(y_test))
print("half data partial model correct: ",partialmodelcounter, " out of ", len(y_test))
        
print("Full model statistics, half data")
print(classification_report(y_test, y_fullmodelpred_list))


print("Partial model statistics, half data")
print(classification_report(y_test, y_partialmodelpred_list))



#train with second half of data set
trained_append_layeroutput.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader2:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizerappendoutput.zero_grad()
        
        y_pred = trained_append_layeroutput(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizerappendoutput.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()


firstlayers_model1.layer_1 = copy.deepcopy(trained_append_layeroutput.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_append_layeroutput.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_append_layeroutput.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_append_layeroutput.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_append_layeroutput.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_append_layeroutput.layer_6)



        
print("Results using full training data:")



#full model predictions
y_fullmodelpred_list =[]
trained_append_layeroutput.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained_append_layeroutput(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]


#partial model predictions
y_partialmodelpred_list =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_partialmodelpred_list.append(y_pred_tag.cpu().numpy())


y_partialmodelpred_list = [a.squeeze().tolist() for a in y_partialmodelpred_list]





#prints correct results
fullmodelcounter=0
partialmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       
    if y_partialmodelpred_list[i]==y_test[i]:
        partialmodelcounter=partialmodelcounter+1
        
print("full data full model correct: ",fullmodelcounter, " out of ", len(y_test))
print("full data partial model correct: ",partialmodelcounter, " out of ", len(y_test))


print("Full model statistics")
print(classification_report(y_test, y_fullmodelpred_list))


print("Partial model statistics")
print(classification_report(y_test, y_partialmodelpred_list))
        






precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
Results using half training data:
half data full model correct:  92  out of  188
half data partial model correct:  100  out of  188
Full model statistics, half data
              precision    recall  f1-score   support

         0.0       0.48      1.00      0.65        88
         1.0       1.00      0.04      0.08       100

    accuracy                           0.49       188
   macro avg       0.74      0.52      0.36       188
weighted avg       0.76      0.49      0.34       188

Partial model statistics, half data
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        88
         1.0       0.53      1.00      0.69       100

    accuracy                           0.53       188
   macro avg       0.27      0.50      0.35       188
weighted avg       0.28      0.53      0.37       188

Results using full training data:
full data full model correct:  104  out o

In [11]:
#add middle layer train with first half then second

#train using first half
trained_extra_inner_layer.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader1:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizermiddle.zero_grad()
        
        y_pred = trained_extra_inner_layer(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizermiddle.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")

print("Results using half training data:")

#copy over first part of model
firstlayers_model1.layer_1 = copy.deepcopy(trained_extra_inner_layer.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_extra_inner_layer.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_extra_inner_layer.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_extra_inner_layer.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_extra_inner_layer.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_extra_inner_layer.layer_out)

#results full model, half data
y_fullmodelpred_list =[]
trained_extra_inner_layer.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained_extra_inner_layer(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]




#results partial model
y_partialmodelpred_list =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_partialmodelpred_list.append(y_pred_tag.cpu().numpy())


y_partialmodelpred_list = [a.squeeze().tolist() for a in y_partialmodelpred_list]




#counts correct values
fullmodelcounter=0
partialmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       
    if y_partialmodelpred_list[i]==y_test[i]:
        partialmodelcounter=partialmodelcounter+1

print("half data full model correct: ",fullmodelcounter, " out of ", len(y_test))
print("half data partial model correct: ",partialmodelcounter, " out of ", len(y_test))
        
print("Full model statistics, half data")
print(classification_report(y_test, y_fullmodelpred_list))


print("Partial model statistics, half data")
print(classification_report(y_test, y_partialmodelpred_list))



#train with second half of data set
trained_extra_inner_layer.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader2:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizermiddle.zero_grad()
        
        y_pred = trained_extra_inner_layer(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizermiddle.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()


firstlayers_model1.layer_1 = copy.deepcopy(trained_extra_inner_layer.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_extra_inner_layer.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_extra_inner_layer.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_extra_inner_layer.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_extra_inner_layer.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_extra_inner_layer.layer_out)



        
print("Results using full training data:")



#full model predictions
y_fullmodelpred_list =[]
trained_extra_inner_layer.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained_extra_inner_layer(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]


#partial model predictions
y_partialmodelpred_list =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_partialmodelpred_list.append(y_pred_tag.cpu().numpy())


y_partialmodelpred_list = [a.squeeze().tolist() for a in y_partialmodelpred_list]





#prints correct results
fullmodelcounter=0
partialmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       
    if y_partialmodelpred_list[i]==y_test[i]:
        partialmodelcounter=partialmodelcounter+1
        
print("full data full model correct: ",fullmodelcounter, " out of ", len(y_test))
print("full data partial model correct: ",partialmodelcounter, " out of ", len(y_test))


print("Full model statistics")
print(classification_report(y_test, y_fullmodelpred_list))


print("Partial model statistics")
print(classification_report(y_test, y_partialmodelpred_list))
        


        



precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
Results using half training data:
half data full model correct:  172  out of  188
half data partial model correct:  100  out of  188
Full model statistics, half data
              precision    recall  f1-score   support

         0.0       0.95      0.86      0.90        88
         1.0       0.89      0.96      0.92       100

    accuracy                           0.91       188
   macro avg       0.92      0.91      0.91       188
weighted avg       0.92      0.91      0.91       188

Partial model statistics, half data
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        88
         1.0       0.53      1.00      0.69       100

    accuracy                           0.53       188
   macro avg       0.27      0.50      0.35       188
weighted avg       0.28      0.53      0.37       188

Results using full training data:
full data full model correct:  172  out 

In [12]:
#append layer, train regularly

normalappend=extra_inner_layer(trained)
normaloptimappend = optim.Adam(normalappend.parameters(), lr=LEARNING_RATE)


normalappend.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        normaloptimappend.zero_grad()
        
        y_pred = normalappend(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        normaloptimappend.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

y_predlist =[]
normalappend.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = normalappend(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_predlist.append(y_pred_tag.cpu().numpy())


y_predlist = [a.squeeze().tolist() for a in y_predlist]



correctcounter=0
for i in range(len(y_predlist)):
    if y_predlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
        
        
print("Trained append model:")
print("Number correct full model: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_predlist))


#copy over first part of model
firstlayers_model1.layer_1 = copy.deepcopy(trained_append_layer.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_append_layer.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_append_layer.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_append_layer.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_append_layer.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_append_layer.layer_6)



firstlayeroptimizer=optim.Adam(firstlayers_model1.parameters(), lr=LEARNING_RATE)
firstlayers_model1.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        firstlayeroptimizer.zero_grad()
        
        y_pred = firstlayers_model1(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        firstlayeroptimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        
        
y_predlist =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_predlist.append(y_pred_tag.cpu().numpy())


y_predlist = [a.squeeze().tolist() for a in y_predlist]



correctcounter=0
for i in range(len(y_predlist)):
    if y_predlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
        
        
print("Tained parial model;")
print("Number correct full model: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_predlist))






Trained append model:
Number correct full model:  179  out of  188
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.95      0.94      0.95        88
         1.0       0.95      0.96      0.96       100

    accuracy                           0.95       188
   macro avg       0.95      0.95      0.95       188
weighted avg       0.95      0.95      0.95       188

Tained parial model;
Number correct full model:  176  out of  188
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93        88
         1.0       0.95      0.93      0.94       100

    accuracy                           0.94       188
   macro avg       0.94      0.94      0.94       188
weighted avg       0.94      0.94      0.94       188



In [13]:
#append layer and output, and train regularly


normalextralayeroutput=append_layer_and_ouput(trained)
normaloptimextraoutput = optim.Adam(normalextralayeroutput.parameters(), lr=LEARNING_RATE)


normalextralayeroutput.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        normaloptimextraoutput.zero_grad()
        
        y_pred = normalextralayeroutput(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        normaloptimextraoutput.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

y_predlist =[]
normalextralayeroutput.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = normalextralayeroutput(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_predlist.append(y_pred_tag.cpu().numpy())


y_predlist = [a.squeeze().tolist() for a in y_predlist]


correctcounter=0
for i in range(len(y_predlist)):
    if y_predlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
print("Trained append layer and output")        
print("Number correct: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_predlist))




firstlayers_model1.layer_1 = copy.deepcopy(trained_append_layeroutput.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_append_layeroutput.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_append_layeroutput.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_append_layeroutput.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_append_layeroutput.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_append_layeroutput.layer_6)




firstlayeroptimizer=optim.Adam(firstlayers_model1.parameters(), lr=LEARNING_RATE)
firstlayers_model1.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        firstlayeroptimizer.zero_grad()
        
        y_pred = firstlayers_model1(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        firstlayeroptimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        
        
y_predlist =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_predlist.append(y_pred_tag.cpu().numpy())


y_predlist = [a.squeeze().tolist() for a in y_predlist]



correctcounter=0
for i in range(len(y_predlist)):
    if y_predlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
        
        
print("Tained parial model;")
print("Number correct full model: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_predlist))






Trained append layer and output
Number correct:  163  out of  188
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.80      0.95      0.87        88
         1.0       0.95      0.79      0.86       100

    accuracy                           0.87       188
   macro avg       0.88      0.87      0.87       188
weighted avg       0.88      0.87      0.87       188

Tained parial model;
Number correct full model:  179  out of  188
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95        88
         1.0       0.96      0.95      0.95       100

    accuracy                           0.95       188
   macro avg       0.95      0.95      0.95       188
weighted avg       0.95      0.95      0.95       188



In [14]:
#just add middle layer, and train regularly


normalextralayer=extra_inner_layer(trained)
normaloptimextra = optim.Adam(normalextralayer.parameters(), lr=LEARNING_RATE)


normalextralayer.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        normaloptimextra.zero_grad()
        
        y_pred = normalextralayer(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        normaloptimextra.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

y_predlist =[]
normalextralayer.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = normalextralayer(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_predlist.append(y_pred_tag.cpu().numpy())


y_predlist = [a.squeeze().tolist() for a in y_predlist]


correctcounter=0
for i in range(len(y_predlist)):
    if y_predlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
print("trained add middle layer")        
print("Number correct: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_predlist))






firstlayers_model1.layer_1 = copy.deepcopy(trained_extra_inner_layer.layer_1)
firstlayers_model1.layer_2 = copy.deepcopy(trained_extra_inner_layer.layer_2)
firstlayers_model1.layer_3 = copy.deepcopy(trained_extra_inner_layer.layer_3)
firstlayers_model1.layer_4 = copy.deepcopy(trained_extra_inner_layer.layer_4)
firstlayers_model1.layer_5 = copy.deepcopy(trained_extra_inner_layer.layer_5)
firstlayers_model1.layer_out = copy.deepcopy(trained_extra_inner_layer.layer_out)



firstlayeroptimizer=optim.Adam(firstlayers_model1.parameters(), lr=LEARNING_RATE)
firstlayers_model1.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        firstlayeroptimizer.zero_grad()
        
        y_pred = firstlayers_model1(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        firstlayeroptimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        
        
y_predlist =[]
firstlayers_model1.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = firstlayers_model1(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_predlist.append(y_pred_tag.cpu().numpy())


y_predlist = [a.squeeze().tolist() for a in y_predlist]



correctcounter=0
for i in range(len(y_predlist)):
    if y_predlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
        
        
print("Tained parial model;")
print("Number correct full model: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_predlist))





trained add middle layer
Number correct:  180  out of  188
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.98      0.93      0.95        88
         1.0       0.94      0.98      0.96       100

    accuracy                           0.96       188
   macro avg       0.96      0.96      0.96       188
weighted avg       0.96      0.96      0.96       188

Tained parial model;
Number correct full model:  179  out of  188
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95        88
         1.0       0.99      0.92      0.95       100

    accuracy                           0.95       188
   macro avg       0.95      0.95      0.95       188
weighted avg       0.95      0.95      0.95       188



In [15]:
#Find and fix weights

#layer 1 weights
#print(GroundTruth.layer_2.weight)


#get weighs for seconds layer
#print(repr(GroundTruth.layer_2.weight.detach().numpy()))

#layer 1 weights
#print(GroundTruth.layer_1.weight)
#print(GroundTruth.layer_2.weight)
#print(GroundTruth.layer_3.weight)
#print(GroundTruth.layer_4.weight)
#print(GroundTruth.layer_5.weight)
#print(GroundTruth.layer_out.weight)



#bais1
#print(GroundTruth.layer_2.bias)


#bais2
#print(GroundTruth.layer_2.bias)


#bais3
#print("BIASES")
#print(GroundTruth.layer_1.bias)
#print(GroundTruth.layer_2.bias)
#print(GroundTruth.layer_3.bias)
#print(GroundTruth.layer_4.bias)
#print(GroundTruth.layer_5.bias)
#print(GroundTruth.layer_out.bias)

#Counts number of ground truths
truthcount=0
for i in y:
    if i==1:
        truthcount=truthcount+1
        
#print(truthcount)
#print(len(y))
#print(len(y_test))
